In [1]:
from ioos_model_comparisons.models import rtofs
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# ds = rtofs().chunks({"time": 40})
# ds
url = "https://tds.marine.rutgers.edu/thredds/dodsC/cool/rtofs/rtofs_us_east_scraped"
ds = xr.open_dataset(url, chunks={"depth": 40, "y": 570, "x": 371, "time": 1})
ds

<xarray.Dataset>
Dimensions:      (depth: 40, y: 1710, x: 742, time: 4635)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 1 2 3 4 5 6 7 ... 1704 1705 1706 1707 1708 1709 1710
  * x            (x) int32 1 2 3 4 5 6 7 8 9 ... 735 736 737 738 739 740 741 742
  * time         (time) datetime64[ns] 2019-07-17T06:00:00 ... 2022-10-04
Data variables:
    date         (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(570, 371), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(570, 371), meta=np.ndarray>
    u            (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 570, 371), meta=np.ndarray>
    v            (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 570, 371), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 570, 371), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 570, 371), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [3]:
extent = [-71, -62, 15, 22]

# t0 = pd.Timestamp(2018, 1, 1, 12, 0, 0)
t0 = pd.Timestamp(2021, 3, 4, 6, 0, 0)
t1 = pd.Timestamp(2022, 8, 4, 6, 0, 0)

In [4]:
dates = pd.date_range(t0, t1, freq='24H')
dates

DatetimeIndex(['2021-03-04 06:00:00', '2021-03-05 06:00:00',
               '2021-03-06 06:00:00', '2021-03-07 06:00:00',
               '2021-03-08 06:00:00', '2021-03-09 06:00:00',
               '2021-03-10 06:00:00', '2021-03-11 06:00:00',
               '2021-03-12 06:00:00', '2021-03-13 06:00:00',
               ...
               '2022-07-26 06:00:00', '2022-07-27 06:00:00',
               '2022-07-28 06:00:00', '2022-07-29 06:00:00',
               '2022-07-30 06:00:00', '2022-07-31 06:00:00',
               '2022-08-01 06:00:00', '2022-08-02 06:00:00',
               '2022-08-03 06:00:00', '2022-08-04 06:00:00'],
              dtype='datetime64[ns]', length=519, freq='24H')

In [5]:
# Save rtofs lon and lat as variables to speed up indexing calculation
grid_lons = ds.lon.values[0,:]
grid_lats = ds.lat.values[:,0]
grid_x = ds.x.values
grid_y = ds.y.values

# Find x, y indexes of the area we want to subset
lons_ind = np.interp(extent[:2], grid_lons, grid_x)
lats_ind = np.interp(extent[2:], grid_lats, grid_y)

# Use np.floor on the 1st index and np.ceil on the 2nd index of each slice 
# in order to widen the area of the extent slightly.
extent_ind = [
    np.floor(lons_ind[0]).astype(int),
    np.ceil(lons_ind[1]).astype(int),
    np.floor(lats_ind[0]).astype(int),
    np.ceil(lats_ind[1]).astype(int)
    ]
# extent_ind

# Use .isel selector on x/y since we know indexes that we want to slice
tds = ds.isel(
    x=slice(extent_ind[0], extent_ind[1]), 
    y=slice(extent_ind[2], extent_ind[3])
)
tds

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 4635)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2019-07-17T06:00:00 ... 2022-10-04
Data variables:
    date         (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    u            (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
    v            (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [6]:
tds_times = tds.sel(time=dates, method='nearest')
tds_times

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 519)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2021-03-04T06:00:00 ... 2022-08-04T06:...
Data variables:
    date         (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    u            (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
    v            (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(1, 40, 94, 8), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [7]:
from dask.diagnostics import ProgressBar

# or distributed.progress when using the distributed scheduler
delayed_obj = tds_times.to_netcdf("manipulated-example-data.nc", compute=False)

with ProgressBar():
    results = delayed_obj.compute()

[                                        ] | 0% Completed | 28.8s


KeyboardInterrupt: 